# Get weight values from .tflite file

- [https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter)

In [35]:
import tensorflow as tf
import numpy as np

In [36]:
print('TensorFlow version: ', tf.__version__)

TensorFlow version:  1.15.0


## Load .tflite file

In [37]:
tfl_path = 'mnist_float.tflite'

In [38]:
interp = tf.lite.Interpreter(model_path=tfl_path)
# , model_content=None
interp.allocate_tensors()

## Show tensor info

- __t__ is list of tensor & type of dict ('name', 'index', 'shape', 'dtype', 'quantization')

In [39]:
for t in interp.get_tensor_details():
    print('name = %s | shape = %s' % (t['name'], t['shape']) )

name = activation/Relu | shape = [  1 256]
name = activation_1/Softmax | shape = [ 1 10]
name = batch_normalization_v1/FusedBatchNorm | shape = [ 1 28 28  1]
name = batch_normalization_v1/FusedBatchNorm_add_param | shape = [1]
name = batch_normalization_v1/FusedBatchNorm_mul_0 | shape = [ 1 28 28  1]
name = batch_normalization_v1/FusedBatchNorm_mul_0_param | shape = [1]
name = batch_normalization_v1_1/FusedBatchNorm | shape = [ 1 14 14 64]
name = batch_normalization_v1_1/FusedBatchNorm_add_param | shape = [64]
name = batch_normalization_v1_1/FusedBatchNorm_mul_0 | shape = [ 1 14 14 64]
name = batch_normalization_v1_1/FusedBatchNorm_mul_0_param | shape = [64]
name = batch_normalization_v1_2/FusedBatchNorm | shape = [  1   7   7 128]
name = batch_normalization_v1_2/FusedBatchNorm_add_param | shape = [128]
name = batch_normalization_v1_2/FusedBatchNorm_mul_0 | shape = [  1   7   7 128]
name = batch_normalization_v1_2/FusedBatchNorm_mul_0_param | shape = [128]
name = batch_normalization_v1

## Show some weight

In [40]:
print(interp.get_tensor(20))

[[[[-1.70315448e-02 -3.58533710e-02  2.25957064e-03 ...  8.54449868e-02
     4.45457324e-02 -3.11175175e-02]
   [ 2.06523808e-03  1.27954560e-03 -1.28422892e-02 ...  4.08943780e-02
    -1.87906120e-02 -1.85506567e-02]
   [-7.36819394e-03 -2.58509461e-02  2.68368050e-02 ... -1.11991018e-02
    -4.42625061e-02 -8.75554141e-03]
   [-1.42701212e-02 -2.31055040e-02 -5.28005557e-03 ...  1.22633129e-02
    -1.18496120e-02  2.64341570e-03]
   [-2.31778622e-02 -2.22251676e-02  2.61159008e-03 ... -3.06947157e-02
    -1.43589480e-02  5.66290598e-03]]

  [[-3.27971466e-02 -4.25918773e-02 -1.06717856e-03 ...  1.99059825e-02
    -4.93862340e-03  2.00388115e-02]
   [ 1.79924984e-02  1.75873656e-02 -2.78173015e-02 ...  2.08080988e-02
     1.19156512e-02  1.31145550e-03]
   [-6.80082885e-05 -4.26534005e-02  9.66546591e-03 ...  9.47579835e-03
    -3.02643375e-03 -1.17767109e-02]
   [-8.98014102e-03 -2.19606869e-02 -2.26806458e-02 ...  3.66818085e-02
    -3.72895203e-03  1.06819859e-03]
   [ 1.55518716e-

***

## Inference

In [41]:
# Get MNIST test data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [42]:
interp.allocate_tensors()
input = interp.tensor(interp.get_input_details()[0]["index"])
output = interp.tensor(interp.get_output_details()[0]["index"])

In [43]:
result = list()
for x in x_test:
    x = np.expand_dims(np.expand_dims(x, axis=0), axis=3)
    interp.set_tensor(interp.get_input_details()[0]['index'], x)
    interp.invoke()
    
    result.append(np.argmax(output()))

In [44]:
corr = np.sum(np.array(result) == y_test)
print('Accuracy: %d / %d' % (corr, len(x_test)))

Accuracy: 9936 / 10000
